In [1]:
import sys
sys.path.append('/../QmeQ/qmeq/')
#sys.path.append('../qmeq/')
import qmeq
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#DEFINING MY FUNCTIONS

def fermi(mu, E, T):
    return 1/(1+np.exp((E-mu)/T))

def calculate_current(epsilon, mu_L, mu_R, gammaL, gammaR, T_L, T_R):

    fL = fermi(mu_L, epsilon, T_L)
    fR = fermi(mu_R, epsilon, T_R)

    W_1_0 = gammaL*fL+gammaR*fR
    W_0_1 = gammaL*(1-fL)+ gammaR*(1-fR)
    Wsum = W_0_1 + W_1_0
    P_0 = W_0_1/Wsum
    P_1 = W_1_0/Wsum

    I = -(gammaR*fR*P_0 - gammaR*(1-fR)*P_1)

    #------------------calculate cumulants---------------------

    I_mean = gammaL*gammaR*(fL-fR)/(gammaL+gammaR)
    I_var =  gammaL*gammaR*((gammaL**2+gammaR**2)*(fL+fR-2*fL*fR)+2*gammaL*gammaR*(fL*(1-fL)+fR*(1-fR)))/((gammaL+gammaR)**3)
    #(Not actually exactly variance apparently...)

    return I, I_mean, I_var

def calculate_heat_current(I, epsilon, mu):
    E = epsilon*I
    Q = E-mu*I
    return Q

In [3]:
#--SETUP: ---------------------------------------------------------------------------------
epsilon = 20
V_B = 30
mu_L = -V_B/2       
mu_R = V_B/2        

gammaL = 0.3
gammaR = gammaL
T_L = 20
T_R = 1

In [4]:
#--------------------QmeQ parameters----------------------------------

tL = np.sqrt(gammaL/np.pi/2)
tR = np.sqrt(gammaR/np.pi/2)

n = 1 #single resonant energy level
h = {(0,0):epsilon}

nleads = 2
mulst = {0:mu_L, 1:mu_R}
tlst = {0:T_L, 1:T_R}
tleads = {(0, 0):tL, (1, 0):tR}

In [5]:
#CALCULATING CURRENT AND NOISE USING QMEQ/MY FUNCTIONS
#------------------------------------------------------------------------------------------

#CALCULATING USING QMEQ---------------------------------------------------------------------
system = qmeq.Builder(nsingle=n, hsingle=h, nleads=nleads,
                         mulst=mulst, tlst=tlst, tleads=tleads, dband=1e4, countingleads=[0], kerntype='pyPauli')

system.solve()
print('QmeQ Standard current:',system.current)
print('QmeQ Counting currents and noise at the left lead (Pauli):', system.current_noise)
print('System sucess:',system.success)
print('')

#CALCULATING USING MY FUNCTIONS-----------------------------------------------------------------
INPUT = {'epsilon': epsilon,         
            'mu_L': mu_L,
            'mu_R': mu_R,
            'gammaL': gammaL,                   # gamma in left lead
            'gammaR': gammaR,                 # gamma in right lead
            'T_L': T_L,          # Temperature in left lead
            'T_R': T_R}                    # Temperature in right lead

I, I_mean, I_var = calculate_current(**INPUT)

print('MY: standard current:',I)
print('MY:Counting currents and noise at the left lead (Pauli):', I_var)


QmeQ Standard current: [ 0.02120315 -0.02120315]
QmeQ Counting currents and noise at the left lead (Pauli): [0.02120315 0.02141517]
System sucess: True

MY: standard current: 0.02120315206611069
MY:Counting currents and noise at the left lead (Pauli): 0.02141517113707885
